In [33]:
import requests
import os

Day = 10

# get file from website using private session key stored in enviromental variables
r = requests.get(
            f'https://adventofcode.com/2023/day/'+str(Day)+'/input',
            cookies={'session': os.getenv('AdventSessionKey')}
)

# read r.text
text_array = r.text.strip().split('\n')


In [34]:
def convert(text_array):
    # Define the directions for each symbol
    directions = {
        "|": ["N", "S"],
        "-": ["E", "W"],
        "L": ["N", "E"],
        "J": ["N", "W"],
        "7": ["S", "W"],
        "F": ["S", "E"]
    }

    # Define the opposite directions
    opposite_directions = {
        "N": "S",
        "E": "W",
        "S": "N",
        "W": "E"
    }

    # Define the movement for each direction
    movement = {"N": (-1, 0), "E": (0, 1), "S": (1, 0), "W": (0, -1)}

    # Find the starting position
    start_row, start_col = next((i, j) for i, row in enumerate(text_array) for j, cell in enumerate(row) if cell == "S")
    
    # Try each direction one by one
    for start_direction in ["N", "E", "S", "W"]:

        # Initialize the number array with the same size as the text array
        number_array = [[None]*len(row) for row in text_array]

        # Assign 0 to the starting position
        number_array[start_row][start_col] = (0, None)
        
        # Start from the 'S' symbol and the current direction
        row, col, direction, distance = start_row, start_col, start_direction, 0

        while True:
            # Move to the next position based on the direction
            d_row, d_col = movement[direction]
            row += d_row
            col += d_col

            # Check if we're out of bounds or have looped back to the 'S' symbol
            if not(0 <= row < len(text_array) and 0 <= col < len(text_array[0])) or (row == start_row and col == start_col):
                break

            # Check the symbol at the new position
            symbol = text_array[row][col]
            if symbol in directions and opposite_directions[direction] in directions[symbol]:
                # Update the distance and the number array
                distance += 1
                number_array[row][col] = (distance, direction)

                # Update the direction to the other connecting direction for the symbol
                direction = next(d for d in directions[symbol] if d != opposite_directions[direction])

        # Check if we have looped back to the 'S' symbol
        if row == start_row and col == start_col:
            return number_array, distance

    # Return Nones if it failed
    return None, None

# Print the result
number_array, intersection_distance = convert(text_array)
#for row in number_array:
#    print([cell[0] if cell is not None else None for cell in row])
print("The distance at the point of intersection is:", int((intersection_distance+1) / 2))
# 6867

The distance at the point of intersection is: 6867
